A Scrapy crawler that uses Selenium to simulate browser scrolling to pull up additional links before scraping.

In [1]:
from selenium import webdriver

from scrapy.http import Request
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
import scrapy
import time

from selenium import webdriver

class NPRSpider(scrapy.Spider):
    name = 'npr-spider'
    allowed_domains = ['www.npr.org']

    def start_requests(self):
        urls = ['https://www.npr.org/people/1936301/john-burnett/archive',
               'https://www.npr.org/people/348780034/nicole-cohen/archive',
               'https://www.npr.org/people/2100689/christopher-joyce/archive',
               'https://www.npr.org/people/384067907/rebecca-hersher/archive',
               'https://www.npr.org/people/2100615/jon-hamilton/archive',
               'https://www.npr.org/people/468930062/jane-greenhalgh/archive',
               'https://www.npr.org/people/536711382/gisele-grayson/archive',
               'https://www.npr.org/people/348732739/adam-cole/archive',
               'https://www.npr.org/people/578890280/rhitu-chatterjee/archive',
               'https://www.npr.org/people/1931801/linda-wertheimer/archive',
               'https://www.npr.org/people/2101212/lakshmi-singh/archive',
               'https://www.npr.org/people/348730771/aarti-shahani/archive',
               'https://www.npr.org/people/676406186/sacha-pfeiffer/archive',
               'https://www.npr.org/people/96022165/yuki-noguchi/archive',
               'https://www.npr.org/people/131724812/scott-neuman/archive',
               'https://www.npr.org/people/467974737/brett-neely/archive',
               'https://www.npr.org/people/2100955/patti-neighmond/archive',
               'https://www.npr.org/people/392602474/domenico-montanaro/archive',
               'https://www.npr.org/people/448294256/sarah-mccammon/archive',
               'https://www.npr.org/people/2100815/jennifer-ludden/archive',
               'https://www.npr.org/people/122805042/tamara-keith/archive',
               'https://www.npr.org/people/384067907/rebecca-hersher/archive',
               'https://www.npr.org/people/2100615/jon-hamilton/archive',
               'https://www.npr.org/people/2100569/richard-gonzales/archive',
               'https://www.npr.org/people/349241810/rose-friedman/archive',
               'https://www.npr.org/people/1930203/ron-elving/archive',
               'https://www.npr.org/people/2100438/debbie-elliott/archive',
               'https://www.npr.org/people/348778932/michaeleen-doucleff/archive',
                'https://www.npr.org/people/7569853/russell-lewis/archive',
                'https://www.npr.org/people/2101154/ari-shapiro/archive',
                'https://www.npr.org/people/5201175/michel-martin/archive',
                'https://www.npr.org/people/348766539/greg-myre/archive',
                'https://www.npr.org/people/473143808/alison-kodjak/archive', 
                'https://www.npr.org/people/2788801/scott-horsley/archive',
                'https://www.npr.org/people/5201175/michel-martin/archive',
                'https://www.npr.org/people/2100631/richard-harris/archive',
                'https://www.npr.org/people/2101154/ari-shapiro/archive',
                'https://www.npr.org/people/146944972/rob-stein/archive',
                'https://www.npr.org/people/544275644/ryan-lucas/archive',
                'https://www.npr.org/people/2101062/philip-reeves/archive',
                'https://www.npr.org/people/279612138/geoff-brumfiel/archive']
        
        for start_url in urls:
            print('\n',start_url)
            yield Request(url = start_url, callback=self.parsencmilllist)

    def parsencmilllist(self, response):
        driver = webdriver.Chrome(executable_path='./chromedriver')
        driver.get(response.url)
        time.sleep(4)
        
        for i in range(1,60):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)
        sel = Selector(text=driver.page_source)

        links = sel.xpath('//li[@class = "audio-tool audio-tool-transcript"]//a/@href').extract()
        print(len(links))
        for link in links:
            yield Request(link,
                          meta={'type': 'Milling'},
                          callback=self.parsencmachine)

    def parsencmachine(self, response):
            
        for article in [response.xpath('//article[@class="story"]')]:
            yield {
                    'text': article.xpath('//div[@class="transcript storytext"]').extract(),#.xpath('a/@href').extract(),
                    'title': article.xpath('//div[@class="storytitle"]/h1[@class = "transcript"]/a/text()').extract(),
                                }

process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'npr_url_text2.json',       # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(NPRSpider)
process.start()
print('Success!')

Success!
